In [54]:
# pip install llama-index llama-index-embeddings-huggingface llama-index-llms-groq gradio transformers
%pip install llama-index chromadb faiss-cpu sentence-transformers gradio tenacity


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 9.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 5.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 6.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/6

In [27]:
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.2 MB/s eta 0:00:00a 0:00:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near f

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [64]:
import os
import gradio as gr
import time
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
from llama_index.core import SimpleDirectoryReader, Settings, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
import openai

# Set environment variable securely
os.environ['GROQ_API_KEY'] = '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'

@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(openai.RateLimitError)
)
def chat_with_retry(self, messages, **kwargs):
    try:
        return self._chat(messages, **kwargs)
    except openai.RateLimitError as e:
        wait_time = int(e.response.headers.get('Retry-After', 60))
        print(f"Rate limit exceeded. Waiting for {wait_time} seconds.")
        time.sleep(wait_time)
        raise

# Replace the original chat method with the retry version
Groq.chat = chat_with_retry

def get_router_query_engine(file_paths: list, llm_choice: str):
    llm_models = {
        "llama3-8b-8192": "llama3-8b-8192",
        "gemma-7b-it": "gemma-7b-it",
        "gemma2-9b-it": "gemma2-9b-it",
        "llama3-70b-8192": "llama3-70b-8192",
        "mixtral-8x7b-32768":"mixtral-8x7b-32768"
    }
    llm = Groq(model=llm_models[llm_choice])
    Settings.llm = llm
    Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

    splitter = SentenceSplitter(chunk_size=1024)
    
    query_engine_tools = []
    for i, file_path in enumerate(file_paths, 1):
        documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
        nodes = splitter.get_nodes_from_documents(documents)

        summary_index = SummaryIndex(nodes)
        vector_index = VectorStoreIndex(nodes)

        summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize", use_async=True)
        vector_query_engine = vector_index.as_query_engine()

        summary_tool = QueryEngineTool.from_defaults(
            query_engine=summary_query_engine,
            description=f"Useful for summarization questions related to document {i}"
        )
        vector_tool = QueryEngineTool.from_defaults(
            query_engine=vector_query_engine,
            description=f"Useful for retrieving specific context from document {i}"
        )
        
        query_engine_tools.extend([summary_tool, vector_tool])

    router_query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=query_engine_tools,
        verbose=True
    )

    return router_query_engine

def process_query(files, llm_choice, query):
    if not files:
        return "Please upload at least one PDF file."
    
    file_paths = [file.name for file in files]
    query_engine = get_router_query_engine(file_paths, llm_choice)
    
    try:
        response = query_engine.query(query)
        return str(response)
    except openai.RateLimitError:
        return "Rate limit exceeded. Please try again in a few minutes."
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio interface
iface = gr.Interface(
    fn=process_query,
    inputs=[
        gr.File(label="Upload PDFs", file_count="multiple"),
        gr.Dropdown(
            choices=[
                "llama3-8b-8192",
                "gemma-7b-it",
                "gemma2-9b-it",
                "llama3-70b-8192",
                "mixtral-8x7b-32768"
            ],
            label="Choose LLM"
        ),
        gr.Textbox(label="Enter your query")
    ],
    outputs=gr.Textbox(label="Response"),
    title="Agentic RAG",
    description="Upload multiple PDFs, select an LLM, and ask questions about specific documents or all documents also can summarize the documents."
)

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7902


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://6d4554091cce373d50.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Selecting query engine 6: The question asks to summarize document 4, and choice 7 is specifically stated as 'Useful for summarization questions related to document 4'..
Selecting query engine 6: This choice is relevant because it is specifically useful for summarization questions related to document 4, which is what the question is asking for..
Selecting query engine 7: This choice is also relevant because it is useful for retrieving specific context from document 4, which is necessary for extracting entities from the same document..
